### Velocity demo
Current approach shares full optimizer state between rounds. Optimizer states are aggregated when received on server end and then shared out such that all clients begin the next round with the aggregated optimizer state.
Limitations: 
- We are not measuring communication from this, but if desired, it should be pretty easy to add
- Currently, we share the entire optimizer state, but when training only a single layer, we would only need to share the state related to that layer. To only share updated layer, modify _aggregate_optimizer_states.

In [1]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple, Union, Callable


import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import copy
import sys
import base64
import pickle
import copy
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader
from flwr.server.strategy import Strategy
import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context, Status, GetParametersRes, Parameters, GetParametersIns, MetricsAggregationFn,NDArrays,Scalar
from flwr.server import ServerApp, ServerConfig, ServerAppComponents 
from flwr.server.strategy import FedAvg, FedProx
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    ParametersRecord,
    parameters_to_ndarrays,
    array_from_numpy
)
from flwr.server.client_manager import ClientManager, SimpleClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = "mps"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()

Training on mps
Flower 1.15.2 / PyTorch 2.6.0


In [ ]:

BATCH_SIZE = 32

def load_datasets(partition_id, num_partitions: int):
    fds = FederatedDataset(dataset="cifar10", partitioners={"train": num_partitions})
    partition = fds.load_partition(partition_id)
    # Divide data on each node: 80% train, 20% test
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)
    pytorch_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    def apply_transforms(batch):
        
        batch["img"] = [pytorch_transforms(img) for img in batch["img"]]
        return batch

    partition_train_test = partition_train_test.with_transform(apply_transforms)
    trainloader = DataLoader(partition_train_test["train"], batch_size=32, shuffle=True)
    valloader = DataLoader(partition_train_test["test"], batch_size=32)
    testset = fds.load_split("test").with_transform(apply_transforms)
    testloader = DataLoader(testset, batch_size=32)
    return trainloader, valloader, testloader

In [ ]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256*4*4, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool3(x)
        x = x.view(-1, 256*4*4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters, trainable_layers=-1):
    """Set model parameters from a list of NumPy arrays."""
    current_state = OrderedDict(net.state_dict())
    
    if trainable_layers == -1:
        # Update all parameters
        params_dict = zip(current_state.keys(), parameters)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        net.load_state_dict(state_dict, strict=True)
    else:
        # Only update the specified layer's parameters
        # Convert current state to numpy arrays
        numpy_state = [param.cpu().numpy() for param in current_state.values()]
        
        # Update the specific indices with new parameters
        numpy_state[trainable_layers*2] = parameters[0]
        numpy_state[trainable_layers*2 + 1] = parameters[1]
        
        # Convert back to torch and update state dict
        for idx, key in enumerate(current_state.keys()):
            current_state[key] = torch.from_numpy(numpy_state[idx])
        
        net.load_state_dict(current_state, strict=True)


def train(net, trainloader, epochs: int, optimizer=None):
    """Train the network on the training set."""
    print(f"training network...")
    criterion = torch.nn.CrossEntropyLoss()
    if optimizer is None:
        optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            images, labels = batch["img"], batch["label"]
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["img"], batch["label"]
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

def freeze_layers(model: torch.nn.Module, trainable_layers: int) -> None:
        """Freeze specified layers of the model."""
        trainable_layers_set = []
        if trainable_layers == -1:
            trainable_layers_set = [-1]
        else:
            trainable_layers_set = [trainable_layers *2, trainable_layers *2 +1]

        for idx, (name, param) in enumerate(model.named_parameters()):
            
            if idx in trainable_layers_set or trainable_layers_set[0] == -1:
                param.requires_grad = True
                print(f"layer index is {idx} and name{name} is trainabe")
            else:
                param.requires_grad = False
                print(f"layer index is {idx} and name{name} is frozen")


def get_parameters_size(params: Parameters) -> int:
    size = sys.getsizeof(params)  # Base size of the dataclass instance
    size += sys.getsizeof(params.tensor_type)  # Size of the string
    size += sys.getsizeof(params.tensors)  # Size of the list container
    size += sum(sys.getsizeof(tensor) for tensor in params.tensors)  # Size of each bytes object
    return size

def serialize_optimizer_state(state_dict):
    """Serialize optimizer state with reduced memory footprint"""
    lightweight_state = {'state': {}, 'param_groups': state_dict['param_groups']}
    
    for k, v in state_dict['state'].items():
        lightweight_state['state'][k] = {
            'exp_avg': v['exp_avg'].clone().detach().cpu().half().numpy(),  # Use half precision
            'exp_avg_sq': v['exp_avg_sq'].clone().detach().cpu().half().numpy(),
            'step': v['step'].item()  # Store as scalar instead of tensor
        }
    
    return base64.b64encode(pickle.dumps(lightweight_state, protocol=pickle.HIGHEST_PROTOCOL)).decode('ascii')

def deserialize_optimizer_state(serialized_state):
    """Deserialize and reconstruct optimizer state"""
    state_dict = pickle.loads(base64.b64decode(serialized_state))
    reconstructed_state = {'state': {}, 'param_groups': state_dict['param_groups']}
    
    for k, v in state_dict['state'].items():
        reconstructed_state['state'][k] = {
            'exp_avg': torch.tensor(v['exp_avg'], dtype=torch.float32),
            'exp_avg_sq': torch.tensor(v['exp_avg_sq'], dtype=torch.float32),
            'step': torch.tensor(v['step'])
        }
    
    return reconstructed_state


In [ ]:

NETWORK_LEN = len(Net().state_dict().keys()) // 2
EPOCHS = 8
NUM_PARTITIONS = 3
NUM_OF_CYCLES  = 1
NUM_OF_FULL_UPDATES_BETWEEN_CYCLES = 2
NUM_OF_ROUNDS = (NUM_OF_CYCLES * NUM_OF_FULL_UPDATES_BETWEEN_CYCLES) + (NUM_OF_CYCLES * NETWORK_LEN *2)
print(f"Number of rounds: {NUM_OF_ROUNDS}")
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}


In [ ]:
from flwr.common import NDArrays, Scalar

def get_evaluate_fn(
    testloader: DataLoader,
    net: torch.nn.Module,
) -> Callable[[int, NDArrays, Dict[str, Scalar]], Optional[Tuple[float, Dict[str, Scalar]]]]:
    """Return an evaluation function for server-side evaluation."""

    def evaluate(
        server_round: int, parameters: NDArrays, config: Dict[str, Scalar]
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Use the entire test set for evaluation."""
        
        # Copy model parameters to avoid modifying the original
        net_copy = copy.deepcopy(net)
        
        # Update model with the latest parameters
        params_dict = zip(net_copy.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        net_copy.load_state_dict(state_dict, strict=True)
        
        net_copy.to(DEVICE)
        net_copy.eval()

        # Test the model
        loss, accuracy = test(net_copy, testloader)
        
        # Return loss and metrics
        return loss, {"accuracy": accuracy}

    return evaluate

In [ ]:
class DropoutClientManager(SimpleClientManager):
    """Custom ClientManager that simulates client dropouts."""
    def __init__(self, dropout_rate: float = 0.4):
        super().__init__()
        self.dropout_rate = dropout_rate

    def sample(
        self,
        num_clients: int,
        min_num_clients: Optional[int] = None,
    ) -> List[ClientProxy]:
        """Sample clients and simulate dropouts."""
        # Get list of clients from parent class
        clients = super().sample(num_clients, min_num_clients)
        
        # Randomly drop clients based on dropout rate
        num_dropouts = int(len(clients) * self.dropout_rate)
        if num_dropouts > 0:
            dropout_indices = np.random.choice(
                len(clients), 
                size=num_dropouts, 
                replace=False
            )
            clients = [c for i, c in enumerate(clients) if i not in dropout_indices]
        
        return clients

# FedAvg with velocity from global model

In [ ]:
from typing import Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg

fed_part_avg_result = {}
fed_part_avg_model_results = {}

class FedPartAvgWithVelocity(Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
        evaluate_fn: Optional[
            Callable[
                [int, NDArrays, dict[str, Scalar]],
                Optional[tuple[float, dict[str, Scalar]]],
            ]
        ] = None,
        on_fit_config_fn: Optional[Callable[[int], dict[str, Scalar]]] = None,
        on_evaluate_config_fn: Optional[Callable[[int], dict[str, Scalar]]] = None,
        accept_failures: bool = True,
        initial_parameters: Optional[Parameters] = None,
        fit_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
        evaluate_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
        inplace: bool = True,
        layer_update_strategy: str = "sequential",
        
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate_fn
        self.on_fit_config_fn = on_fit_config_fn
        self.on_evaluate_config_fn = on_evaluate_config_fn
        self.accept_failures = accept_failures
        self.initial_parameters = initial_parameters
        self.fit_metrics_aggregation_fn = fit_metrics_aggregation_fn
        self.evaluate_metrics_aggregation_fn = evaluate_metrics_aggregation_fn
        self.inplace = inplace

        self.layer_update_strategy = layer_update_strategy  # 'sequential' or 'cyclic'
        self.current_layer = 0  # Track which layer to update
        self.number_of_layers = None
        self.layer_training_sequence = []
        self.training_sequence_index = 0
        self.latest_parameters = initial_parameters
        self.updated_layers = -1
        self.global_optim_state = None


    def __repr__(self) -> str:
        return "FedPartAvgWithVelocity"
    

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients
    
    def generate_layer_training_sequence(self) -> List[int]:
        """Generate a sequence of layers to train."""
        layer_training_sequence = []
        for _ in range(NUM_OF_CYCLES):
            for _ in range(NUM_OF_FULL_UPDATES_BETWEEN_CYCLES):
                    layer_training_sequence.append(-1)
            for layer in range(NETWORK_LEN):
                    layer_training_sequence.append(layer)
                    layer_training_sequence.append(layer)

        return layer_training_sequence

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        self.layer_training_sequence = self.generate_layer_training_sequence()
        self.number_of_layers = len(ndarrays)
        self.latest_parameters = ndarrays_to_parameters(ndarrays)
        self.latest_parameters = ndarrays_to_parameters(ndarrays)

        return ndarrays_to_parameters(ndarrays)
    


    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[tuple[float, dict[str, Scalar]]]:
        """Evaluate model parameters using an evaluation function."""
        if self.evaluate_fn is None:
            # No evaluation function provided
            return None
        parameters_ndarrays = parameters_to_ndarrays(parameters)
        eval_res = self.evaluate_fn(server_round, parameters_ndarrays, {})
        if eval_res is None:
            return None
        
        if server_round in fed_part_avg_model_results:  
            expand_fed_part_avg_model_results= {**fed_part_avg_model_results[server_round], "global_loss": eval_res[0], "global_metrics": eval_res[1]}
        else:
            expand_fed_part_avg_model_results= {"global_loss": eval_res[0], "global_metrics": eval_res[1]}
        
        fed_part_avg_model_results[server_round] = expand_fed_part_avg_model_results
        
        loss, metrics = eval_res
        return loss, metrics

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""
        
        config = {"trainable_layers": self.layer_training_sequence[self.training_sequence_index], "updated_layers": self.updated_layers}

        if self.global_optim_state is not None:
            config["optimizer_state"] = self.global_optim_state
        
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )
        
        print(f"Training on layer {self.layer_training_sequence}")
        fit_configurations = []

        params_array = parameters_to_ndarrays(parameters)
        
        # If doing full model update, send all parameters
        if self.layer_training_sequence[self.training_sequence_index] == -1 or self.updated_layers == -1:
            selected_params = parameters
        else:
            layer_idx = self.updated_layers
            selected_params = ndarrays_to_parameters([
                    params_array[layer_idx * 2],     # Weight
                    params_array[layer_idx * 2 + 1]  # Bias
                ])

        for idx, client in enumerate(clients):
            fit_configurations.append((client, FitIns(selected_params, config)))

        self.updated_layers = self.layer_training_sequence[self.training_sequence_index]
        self.training_sequence_index = self.training_sequence_index + 1
        
        return fit_configurations
    

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        global global_all_optimizer_states
        """Aggregate fit results using weighted average."""
        
        total_size = 0
        for client, fit_res in results:
            total_size += get_parameters_size(fit_res.parameters)
            total_size += fit_res.metrics["recieved_parameter_size"]

        print(f"total size: {total_size}")
        
        if fed_part_avg_result.get(server_round):
            fed_part_avg_result[server_round]["total_size"] = total_size
        else:
            fed_part_avg_result[server_round] = {"total_size": total_size}
        


        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]

        aggregated_weights = aggregate(weights_results)
        trained_layer = results[0][1].metrics["trained_layer"]
        print(f"aggregated weight size {len(aggregated_weights)} ")

        if trained_layer == -1:
            self.latest_parameters = ndarrays_to_parameters(aggregated_weights)
        else:
            current_model = parameters_to_ndarrays(self.latest_parameters)
            print(f"updateing layers {trained_layer* 2}  and {trained_layer* 2 + 1} ")
            current_model[trained_layer* 2] = aggregated_weights[0]
            current_model[trained_layer* 2 +1] = aggregated_weights[1]
            self.latest_parameters = ndarrays_to_parameters(current_model)

        # include optimizer state in metrics
        metrics = {}
        optimizer_states_serialized = [res.metrics.get("optimizer_state", None) for _, res in results]
        optimizer_states = [deserialize_optimizer_state(state) for state in optimizer_states_serialized if state is not None]
        aggregated_optimizer_state = self._aggregate_optimizer_states(optimizer_states)
        aggregated_optimizer_state_serialized = serialize_optimizer_state(aggregated_optimizer_state)
        metrics["optimizer_state"] = aggregated_optimizer_state_serialized
        self.global_optim_state = aggregated_optimizer_state_serialized

        return self.latest_parameters, metrics


    def _aggregate_optimizer_states(self, optimizer_states):
        # aggregates by comuting (unweighted) mean of 1st and 2nd momentum, and taking the maximum step value. (alternatively, could be changed to reset steps to 0)
        """Aggregate optimizer states with reduced memory usage"""
        if not optimizer_states:
            return None
            
        # Start with an empty state dict structure
        aggregated_state_dict = {'state': {}, 'param_groups': optimizer_states[0]['param_groups']}
        
        # Get all layer keys
        all_layers = list(optimizer_states[0]['state'].keys())
        
        for layer in all_layers:
            # Initialize with zeros instead of stacking
            if layer not in aggregated_state_dict['state']:
                aggregated_state_dict['state'][layer] = {}
                
            # Get shape from first optimizer
            exp_avg_shape = optimizer_states[0]['state'][layer]['exp_avg'].shape
            exp_avg_sq_shape = optimizer_states[0]['state'][layer]['exp_avg_sq'].shape
            
            # Pre-allocate tensors
            sum_exp_avg = torch.zeros(exp_avg_shape, device='cpu')
            sum_exp_avg_sq = torch.zeros(exp_avg_sq_shape, device='cpu')
            max_step = 0
            
            # Sum without stacking
            for state in optimizer_states:
                if layer in state['state']:
                    sum_exp_avg += state['state'][layer]['exp_avg'].cpu()
                    sum_exp_avg_sq += state['state'][layer]['exp_avg_sq'].cpu()
                    max_step = max(max_step, state['state'][layer]['step'].item())
            
            # Average
            n_states = len(optimizer_states)
            aggregated_state_dict['state'][layer] = {
                'exp_avg': sum_exp_avg / n_states,
                'exp_avg_sq': sum_exp_avg_sq / n_states,
                'step': torch.tensor(max_step)
            }
        
        return aggregated_state_dict


    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}
        
        total_loss = 0
        for _, evaluate_res in results:
            total_loss += evaluate_res.loss

        if fed_part_avg_result.get(server_round):
            fed_part_avg_result[server_round]["total_loss"] = total_loss
        else:
            fed_part_avg_result[server_round] = {"total_loss": total_loss}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    

In [ ]:
class FedAvgPartWithVelocityFlowerClient(NumPyClient):
    def __init__(self, partition_id, net, trainloader, valloader, context: Context):
        self.partition_id = partition_id
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.client_state = context.state
    
        # Initialize parameters record if it doesn't exist
        if "net_parameters" not in self.client_state.parameters_records:
            self.client_state.parameters_records["net_parameters"] = ParametersRecord()
            # Save initial model state
            self._save_model_state()


    def get_parameters(self, config):
        print(f"[Client {self.partition_id}] get_parameters")
        parameters = get_parameters(self.net)
        if config["trainable_layers"] == -1:
            trained_layers = parameters
        else:
            trained_layers = [
                parameters[config["trainable_layers"] * 2],
                parameters[config["trainable_layers"] * 2 + 1]
            ]
        
        return trained_layers

    def _save_model_state(self):
        """Save current model parameters to context"""
        p_record = ParametersRecord()
        parameters = get_parameters(self.net)
        
        for i, param in enumerate(parameters):
            p_record[f"layer_{i}"] = array_from_numpy(param)
        
        self.client_state.parameters_records["net_parameters"] = p_record

    def _load_model_state(self):
        """Load model parameters from context"""
        p_record = self.client_state.parameters_records["net_parameters"]
        parameters = []
        
        for i in range(len(p_record)):
            parameters.append(p_record[f"layer_{i}"].numpy())
        
        set_parameters(self.net, parameters)

    def fit(self, parameters, config):
        print(f"[Client {self.partition_id}] fit")

        self._load_model_state()
        recieved_parameter_size = get_parameters_size(ndarrays_to_parameters(parameters))  
        set_parameters(self.net, parameters, config["updated_layers"])
        freeze_layers(self.net, config["trainable_layers"])

        self.optimizer = torch.optim.Adam(self.net.parameters())
        if "optimizer_state" in config:
            print(f"Got optimizer state in config, setting..")
            serialized_optimizer_state = config["optimizer_state"]
            optim_state = deserialize_optimizer_state(serialized_optimizer_state)
            self._set_optimizer_state(optim_state)

        train(self.net, self.trainloader, epochs=EPOCHS, optimizer=self.optimizer)
        self._save_model_state()
        print('finished train')

        # handle optimizer state (serialize before passing)
        optim_state = self._get_optimizer_state()
        serialized_optimizer_state = serialize_optimizer_state(optim_state)
        print(f"After training, got optim state {serialized_optimizer_state[:50]}")

        new_config = {
            "trained_layer":config["trainable_layers"], 
            "recieved_parameter_size": recieved_parameter_size,
            "optimizer_state": serialized_optimizer_state
        }

        return self.get_parameters(config), len(self.trainloader), new_config

    def evaluate(self, parameters, config):
        print(f"[Client {self.partition_id}] evaluate, config: {config}")
        self._load_model_state()
        # This part looks a bit sus. Why are we loading and then setting params to be the same?
        current_state = get_parameters(self.net)
        set_parameters(self.net, current_state)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

    def _get_optimizer_state(self):
        """Extract only the necessary parts of optimizer state"""
        state_dict = self.optimizer.state_dict()
        # Extract only essential information
        pruned_state = {'state': {}, 'param_groups': state_dict['param_groups']}
        
        for k, v in state_dict['state'].items():
            pruned_state['state'][k] = {
                'exp_avg': v['exp_avg'].clone().detach().cpu(),
                'exp_avg_sq': v['exp_avg_sq'].clone().detach().cpu(),
                'step': v['step']
            }
        
        return pruned_state

    def _set_optimizer_state(self, state_dict):
        """Load optimizer state efficiently"""
        # Move tensors to the appropriate device first
        for k in state_dict['state']:
            for key in ['exp_avg', 'exp_avg_sq']:
                if key in state_dict['state'][k]:
                    state_dict['state'][k][key] = state_dict['state'][k][key].to(DEVICE)
        
        self.optimizer.load_state_dict(state_dict)
    

def client_fn(context: Context) -> Client:
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]

    # Initialize network if not in context
    if not hasattr(context, 'net'):
        context.net = Net().to(DEVICE)
    
    trainloader, valloader, _ = load_datasets(partition_id, num_partitions)

    return FedAvgPartWithVelocityFlowerClient(partition_id, context.net, trainloader, valloader, context).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [ ]:
net = Net().to(DEVICE)

_, _, testloader = load_datasets(0, NUM_PARTITIONS)

evaluate_fn = get_evaluate_fn(testloader, net)
client_manager = DropoutClientManager(dropout_rate=0.5)

def server_fn(context: Context) -> ServerAppComponents:
    # Configure the server for just 3 rounds of training
    config = ServerConfig(num_rounds=NUM_OF_ROUNDS)
    return ServerAppComponents(
        config=config,
        strategy=FedPartAvgWithVelocity(
            evaluate_fn=evaluate_fn,
        ),
        client_manager=client_manager
    )

server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

In [ ]:
with open(f'results/fed_part_avg_client_dropout_result.p', 'wb') as file:
    pickle.dump(fed_part_avg_result, file)

with open(f'results/fed_part_avg_model_client_dropout_results.p', 'wb') as file:
    pickle.dump(fed_part_avg_model_results, file)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np



# # Plot the total size of parameters for each round
# fed_part_avg_rounds = list(fed_part_avg_result.keys())
# fed_part_avg_sizes = [fed_part_avg_result[round]["total_size"] for round in fed_part_avg_rounds]

# plt.figure(figsize=(10, 5))
# plt.plot(fed_part_avg_rounds, fed_part_avg_sizes, marker='o', linestyle='-', color='b', label='FedPartAvgMomentum')
# # plt.plot(fed_avg_rounds, fed_avg_sizes, marker='o', linestyle='-', color='r', label='FedAvg')
# plt.xlabel('Round')
# plt.ylabel('Total Size of Parameters (bytes)')
# plt.title('Total Size of Parameters for Each Round')
# plt.legend()
# plt.grid(True)

# fed_part_avg_losses = [fed_part_avg_result[round]["total_loss"] for round in fed_part_avg_rounds]

# plt.figure(figsize=(10, 5))
# plt.plot(fed_part_avg_rounds, fed_part_avg_losses, marker='o', linestyle='-', color='b', label='FedPartAvgMomentum')
# # plt.plot(fed_avg_rounds, fed_avg_losses, marker='o', linestyle='-', color='r', label='FedAvg')
# plt.xlabel('Round')
# plt.ylabel('Total Loss')
# plt.title('Total Loss for Each Round')
# plt.legend()
# plt.grid(True)


# fed_part_avg_model_rounds = list(fed_part_avg_model_results.keys())
# fed_part_avg_accuracies = [fed_part_avg_model_results[round]["global_metrics"]["accuracy"] for round in fed_part_avg_model_rounds]

# plt.figure(figsize=(10, 5))
# plt.plot(fed_part_avg_model_rounds, fed_part_avg_accuracies, marker='o', linestyle='-', color='b', label='FedPartAvgMomentum')
# # plt.plot(fed_avg_model_rounds, fed_avg_accuracies, marker='o', linestyle='-', color='r', label='FedAvg')
# plt.xlabel('Round')
# plt.ylabel('Accuracy')
# plt.title('Accuracy for Each Round')
# plt.legend()
# plt.grid(True)

# fed_part_avg_global_losses = [fed_part_avg_model_results[round]["global_loss"] for round in fed_part_avg_model_rounds]

# plt.figure(figsize=(10, 5))
# plt.plot(fed_part_avg_model_rounds, fed_part_avg_global_losses, marker='o', linestyle='-', color='b', label='FedPartAvgMomentum')
# # plt.plot(fed_avg_model_rounds, fed_avg_global_losses, marker='o', linestyle='-', color='r', label='FedAvg')
# plt.xlabel('Round')
# plt.ylabel('Loss')
# plt.legend()
# plt.title('Loss for Each Round')


# Using momentum similarity for weighing updates
Weigh updates by cosine similarity between client momentum and global momentum from previous round. 

Future work ideas:
- Save cosine similarities. Use these to compute the distributions of these similarities and identify
- Use cosine similarities to identify outlier clients.
This is likely of greater interest in heterogeneous data settings.

In [ ]:
from typing import Union
from functools import partial, reduce
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg

fed_momentum_weighting_results = {}
fed_momentum_weighting_model_results = {}

global client_opt
global prev_opt

class FedPartAvgWithVelocityweighting(Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
        evaluate_fn: Optional[
            Callable[
                [int, NDArrays, dict[str, Scalar]],
                Optional[tuple[float, dict[str, Scalar]]],
            ]
        ] = None,
        on_fit_config_fn: Optional[Callable[[int], dict[str, Scalar]]] = None,
        on_evaluate_config_fn: Optional[Callable[[int], dict[str, Scalar]]] = None,
        accept_failures: bool = True,
        initial_parameters: Optional[Parameters] = None,
        fit_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
        evaluate_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
        inplace: bool = True,
        layer_update_strategy: str = "sequential",
        
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate_fn
        self.on_fit_config_fn = on_fit_config_fn
        self.on_evaluate_config_fn = on_evaluate_config_fn
        self.accept_failures = accept_failures
        self.initial_parameters = initial_parameters
        self.fit_metrics_aggregation_fn = fit_metrics_aggregation_fn
        self.evaluate_metrics_aggregation_fn = evaluate_metrics_aggregation_fn
        self.inplace = inplace

        self.layer_update_strategy = layer_update_strategy  # 'sequential' or 'cyclic'
        self.current_layer = 0  # Track which layer to update
        self.number_of_layers = None
        self.layer_training_sequence = []
        self.training_sequence_index = 0
        self.latest_parameters = initial_parameters
        self.updated_layers = -1
        self.global_optim_state = None


    def __repr__(self) -> str:
        return "FedPartAvgWithVelocityWeighting"
    

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients
    
    def generate_layer_training_sequence(self) -> List[int]:
        """Generate a sequence of layers to train."""
        layer_training_sequence = []
        for _ in range(NUM_OF_CYCLES):
            for _ in range(NUM_OF_FULL_UPDATES_BETWEEN_CYCLES):
                    layer_training_sequence.append(-1)
            for layer in range(NETWORK_LEN):
                    layer_training_sequence.append(layer)
                    layer_training_sequence.append(layer)

        return layer_training_sequence

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        self.layer_training_sequence = self.generate_layer_training_sequence()
        self.number_of_layers = len(ndarrays)

        return ndarrays_to_parameters(ndarrays)
    


    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[tuple[float, dict[str, Scalar]]]:
        """Evaluate model parameters using an evaluation function."""
        if self.evaluate_fn is None:
            # No evaluation function provided
            return None
        parameters_ndarrays = parameters_to_ndarrays(parameters)
        eval_res = self.evaluate_fn(server_round, parameters_ndarrays, {})
        if eval_res is None:
            return None
        
        if server_round in fed_momentum_weighting_model_results:  
            expand_fed_part_avg_model_results= {**fed_momentum_weighting_model_results[server_round], "global_loss": eval_res[0], "global_metrics": eval_res[1]}
        else:
            expand_fed_part_avg_model_results= {"global_loss": eval_res[0], "global_metrics": eval_res[1]}
        
        fed_momentum_weighting_model_results[server_round] = expand_fed_part_avg_model_results
        
        loss, metrics = eval_res
        return loss, metrics

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""
        
        config = {"trainable_layers": self.layer_training_sequence[self.training_sequence_index], "updated_layers": self.updated_layers}

        if self.global_optim_state is not None:
            config["optimizer_state"] = self.global_optim_state
        
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )
        
        print(f"Training on layer {self.layer_training_sequence}")
        fit_configurations = []

        params_array = parameters_to_ndarrays(parameters)
        
        # If doing full model update, send all parameters
        if self.layer_training_sequence[self.training_sequence_index] == -1 or self.updated_layers == -1:
            selected_params = parameters
        else:
            layer_idx = self.updated_layers
            selected_params = ndarrays_to_parameters([
                    params_array[layer_idx * 2],     # Weight
                    params_array[layer_idx * 2 + 1]  # Bias
                ])

        for idx, client in enumerate(clients):
            fit_configurations.append((client, FitIns(selected_params, config)))

        self.updated_layers = self.layer_training_sequence[self.training_sequence_index]
        self.training_sequence_index = self.training_sequence_index + 1
        
        return fit_configurations
    

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        global global_all_optimizer_states
        """Aggregate fit results using weighted average."""
        
        total_size = 0
        for client, fit_res in results:
            total_size += get_parameters_size(fit_res.parameters)
            total_size += fit_res.metrics["recieved_parameter_size"]
            
        print(f"total size: {total_size}")
        
        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]

        metrics = {}
        optimizer_states_serialized = [res.metrics.get("optimizer_state", None) for _, res in results]
        optimizer_states = [deserialize_optimizer_state(state) for state in optimizer_states_serialized if state is not None]

        # Get cosine similarities
        if server_round == 1:
            # First round doesn't have a previous state. Use average momentum instead
            equal_weights = [1] * len(optimizer_states)
            prev_global_optimizer_state = self._aggregate_optimizer_states_weighted(optimizer_states, equal_weights)
        else:
            prev_global_optimizer_state = deserialize_optimizer_state(self.global_optim_state)
        cos_similarities = self._cos_similarity_from_optimizer_states(optimizer_states, prev_global_optimizer_state)

        # Aggregate nn params using weighting from momentum
        aggregated_weights = self._aggregate_params_weighted(weights_results, cos_similarities)
        trained_layer = results[0][1].metrics["trained_layer"] 

        if trained_layer == -1:
            self.latest_parameters = ndarrays_to_parameters(aggregated_weights)
        else:
            current_model = parameters_to_ndarrays(self.latest_parameters)
            current_model[trained_layer* 2] = aggregated_weights[0]
            current_model[trained_layer* 2 +1] = aggregated_weights[1]
            self.latest_parameters = ndarrays_to_parameters(current_model)

        # aggregate optimizer state using weighting from momentum
        aggregated_optimizer_state = self._aggregate_optimizer_states_weighted(optimizer_states, cos_similarities)
        aggregated_optimizer_state_serialized = serialize_optimizer_state(aggregated_optimizer_state)
        metrics["optimizer_state"] = aggregated_optimizer_state_serialized
        self.global_optim_state = aggregated_optimizer_state_serialized

        # Add sizes to total_size:
        # Note that we currently add the whole optimizer state for ease of implementation, but this is redundant for all but the unfrozen layer as the frozen layers opt state is unchanged.
        sizes = sum([sys.getsizeof(data) for data in optimizer_states_serialized if data is not None])
        print(f"Adding optimizer state adds size {sizes}")
        total_size += sizes

        if fed_momentum_weighting_results.get(server_round):
            fed_momentum_weighting_results[server_round]["total_size"] = total_size
        else:
            fed_momentum_weighting_results[server_round] = {"total_size": total_size}

        return self.latest_parameters, metrics


    def _momentum_based_aggregate(results: list[tuple[NDArrays, int]]) -> NDArrays:
        """Compute weighted average of params."""
        num_examples_total = sum(num_examples for (_, num_examples) in results)

        # list of weights, each multiplied by the related number of examples
        weighted_weights = [
            [layer * num_examples for layer in weights] for weights, num_examples in results
        ]

        # Compute average weights of each layer
        weights_prime: NDArrays = [
            reduce(np.add, layer_updates) / num_examples_total
            for layer_updates in zip(*weighted_weights)
        ]
        return weights_prime

    def _aggregate_optimizer_states_weighted(self, optimizer_states, similarities):
        # aggregates by comuting mean of 1st and 2nd momentum, and taking the maximum step value. (alternatively, could be changed to reset steps to 0)
        # uses direct normalization (weight of similarity divided by sum of similarities)
        aggregated_state_dict = copy.deepcopy(optimizer_states[0])
        sum_similarities = sum(similarities)

        for layer in range(len(optimizer_states[0]['state'])):
            weighted_fst_momentum = torch.zeros_like(optimizer_states[0]['state'][layer]['exp_avg'])
            weighted_snd_momentum = torch.zeros_like(optimizer_states[0]['state'][layer]['exp_avg_sq'])
            
            # Accumulate weighted momentums from each client
            for i in range(len(optimizer_states)):
                weight = similarities[i] / sum_similarities
                weighted_fst_momentum += optimizer_states[i]['state'][layer]['exp_avg'] * weight
                weighted_snd_momentum += optimizer_states[i]['state'][layer]['exp_avg_sq'] * weight

            aggregated_state_dict['state'][layer]['exp_avg'] = weighted_fst_momentum
            aggregated_state_dict['state'][layer]['exp_avg_sq'] = weighted_snd_momentum
        max_steps = max([optimizer_states[i]['state'][layer]['step'] for i in range(len(optimizer_states))])
        aggregated_state_dict['state'][layer]['step'] = max_steps
        
        return aggregated_state_dict

    def _aggregate_params_weighted(self, results: list[tuple[NDArrays, int]], cos_similarities: list[float]) -> NDArrays:
        """Compute cos similarity weighted average."""
        # Calculate the total number of examples used during training
        assert len(results) == len(cos_similarities)
        sum_sim = sum(cos_similarities)

        # Create a list of weights, each multiplied by the cos similiarity
        weighted_weights = [
            [layer * cos_similarities[i] for layer in results[i][0]] for i in range(len(results))
        ]

        # Compute average weights of each layer
        weights_prime: NDArrays = [
            reduce(np.add, layer_updates) / sum_sim
            for layer_updates in zip(*weighted_weights)
        ]
        return weights_prime

    def _cos_similarity_from_optimizer_states(self, optimizer_states, prev_global_optimizer_state):
        # Returns: cosine similarity from each optimizer state to the prev global state
        cos = torch.nn.CosineSimilarity(dim=-1)
        similarities = []
        layers = range(len(optimizer_states[0]['state']))

        # get global opt params as vector
        fst_momentum_glob = [prev_global_optimizer_state['state'][l]['exp_avg'].view(-1) for l in layers]
        snd_momentum_glob = [prev_global_optimizer_state['state'][l]['exp_avg_sq'].view(-1) for l in layers]
        glob_state_vec = torch.cat(fst_momentum_glob + snd_momentum_glob)

        for client in range(len(optimizer_states)):
            # get client opt params as vector
            fst_momentum_client = [optimizer_states[client]['state'][l]['exp_avg'].view(-1) for l in layers]
            snd_momentum_client = [optimizer_states[client]['state'][l]['exp_avg_sq'].view(-1) for l in layers]
            client_state_vec = torch.cat(fst_momentum_client + snd_momentum_client)

            sim = cos(client_state_vec, glob_state_vec)
            similarities.append(sim.item())

        return similarities

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}
        
        total_loss = 0
        for _, evaluate_res in results:
            total_loss += evaluate_res.loss

        if fed_momentum_weighting_results.get(server_round):
            fed_momentum_weighting_results[server_round]["total_loss"] = total_loss
        else:
            fed_momentum_weighting_results[server_round] = {"total_loss": total_loss}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

In [ ]:
class FedAvgPartWithVelocityWeightingFlowerClient(NumPyClient):
    def __init__(self, partition_id, net, trainloader, valloader, context: Context):
        self.partition_id = partition_id
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.client_state = context.state

        # Initialize parameters record if it doesn't exist
        if "net_parameters" not in self.client_state.parameters_records:
            self.client_state.parameters_records["net_parameters"] = ParametersRecord()
            # Save initial model state
            self._save_model_state()

    def _save_model_state(self):
        """Save current model parameters to context"""
        p_record = ParametersRecord()
        parameters = get_parameters(self.net)
        
        for i, param in enumerate(parameters):
            p_record[f"layer_{i}"] = array_from_numpy(param)
        
        self.client_state.parameters_records["net_parameters"] = p_record

    def _load_model_state(self):
        """Load model parameters from context"""
        p_record = self.client_state.parameters_records["net_parameters"]
        parameters = []
        
        for i in range(len(p_record)):
            parameters.append(p_record[f"layer_{i}"].numpy())
        
        set_parameters(self.net, parameters)

    def get_parameters(self, config):
        print(f"[Client {self.partition_id}] get_parameters")
        parameters = get_parameters(self.net)
        trainable_layer = config["trainable_layers"]
        self._save_model_state()
        
        if trainable_layer == -1:
            return parameters
        
        trained_layer = [parameters[trainable_layer*2], parameters[trainable_layer*2 +1]]
        return trained_layer

    def fit(self, parameters, config):
        print(f"[Client {self.partition_id}] fit")

        self._load_model_state()
        received_parameter_size = get_parameters_size(ndarrays_to_parameters(parameters))
        set_parameters(self.net, parameters, config["updated_layers"])
        freeze_layers(self.net, config["trainable_layers"])

        self.optimizer = torch.optim.Adam(self.net.parameters())
        if "optimizer_state" in config:
            print(f"Got optimizer state in config, setting..")
            serialized_optimizer_state = config["optimizer_state"]
            optim_state = deserialize_optimizer_state(serialized_optimizer_state)
            self._set_optimizer_state(optim_state)

        train(self.net, self.trainloader, epochs=EPOCHS, optimizer=self.optimizer)
        print('finished train')

        # handle optimizer state (serialize before passing)
        optim_state = self._get_optimizer_state()
        serialized_optimizer_state = serialize_optimizer_state(optim_state)
        print(f"After training, got optim state {serialized_optimizer_state[:50]}")

        new_config =  {
            "trained_layer":config["trainable_layers"], 
            "recieved_parameter_size": received_parameter_size,
            "optimizer_state": serialized_optimizer_state
            }

        self._save_model_state()

        return self.get_parameters(config), len(self.trainloader), new_config

    def evaluate(self, parameters, config):
        print(f"[Client {self.partition_id}] evaluate, config: {config}")
        self._load_model_state()
        current_state = get_parameters(self.net)
        set_parameters(self.net, current_state)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

    
    def _get_optimizer_state(self):
        state_dict = self.optimizer.state_dict()
        return state_dict
    
    def _set_optimizer_state(self, state_dict):
        self.optimizer.load_state_dict(state_dict)


def client_fn(context: Context) -> Client:
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]

    # Initialize network if not in context
    if not hasattr(context, 'net'):
        context.net = Net().to(DEVICE)

    trainloader, valloader, _ = load_datasets(partition_id, num_partitions)
    return FedAvgPartWithVelocityWeightingFlowerClient(partition_id, context.net, trainloader, valloader, context).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [ ]:
net = Net().to(DEVICE)

_, _, testloader = load_datasets(0, NUM_PARTITIONS)

evaluate_fn = get_evaluate_fn(testloader, net)
client_manager = DropoutClientManager(dropout_rate=0.5)

def server_fn(context: Context) -> ServerAppComponents:
    # Configure the server for just 3 rounds of training
    config = ServerConfig(num_rounds=NUM_OF_ROUNDS)
    return ServerAppComponents(
        config=config,
        strategy=FedPartAvgWithVelocityweighting(
            evaluate_fn=evaluate_fn,
        ),
        client_manager=client_manager
    )

server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

In [ ]:
with open(f'results/fed_momentum_weighting_client_dropout_results.p', 'wb') as file:
    pickle.dump(fed_momentum_weighting_results, file)

with open(f'results/fed_momentum_weighting_model_client_dropout_results.p', 'wb') as file:
    pickle.dump(fed_momentum_weighting_model_results, file)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Plot the total size of parameters for each round
# fed_momentum_weighting_rounds = list(fed_momentum_weighting_results.keys())
# fed_momentum_weighting_sizes = [fed_momentum_weighting_results[round]["total_size"] for round in fed_momentum_weighting_rounds]

# plt.figure(figsize=(10, 5))
# plt.plot(fed_momentum_weighting_rounds, fed_momentum_weighting_sizes, marker='o', linestyle='-', color='b', label='FedPartWithVelocityWeighting')
# # plt.plot(fed_avg_rounds, fed_avg_sizes, marker='o', linestyle='-', color='r', label='FedAvg')
# plt.xlabel('Round')
# plt.ylabel('Total Size of Parameters (bytes)')
# plt.title('Total Size of Parameters for Each Round')
# plt.legend()
# plt.grid(True)

# fed_momentum_weighting_losses = [fed_momentum_weighting_results[round]["total_loss"] for round in fed_momentum_weighting_rounds]

# plt.figure(figsize=(10, 5))
# plt.plot(fed_momentum_weighting_rounds, fed_momentum_weighting_losses, marker='o', linestyle='-', color='b', label='FedPartWithVelocityWeighting')
# # plt.plot(fed_avg_rounds, fed_avg_losses, marker='o', linestyle='-', color='r', label='FedAvg')
# plt.xlabel('Round')
# plt.ylabel('Total Loss')
# plt.title('Total Loss for Each Round')
# plt.legend()
# plt.grid(True)


# fed_momentum_weighting_model_rounds = list(fed_momentum_weighting_model_results.keys())
# fed_momentum_weighting_accuracies = [fed_momentum_weighting_model_results[round]["global_metrics"]["accuracy"] for round in fed_momentum_weighting_model_rounds]

# plt.figure(figsize=(10, 5))
# plt.plot(fed_momentum_weighting_model_rounds, fed_momentum_weighting_accuracies, marker='o', linestyle='-', color='b', label='FedPartWithVelocityWeighting')
# # plt.plot(fed_avg_model_rounds, fed_avg_accuracies, marker='o', linestyle='-', color='r', label='FedAvg')
# plt.xlabel('Round')
# plt.ylabel('Accuracy')
# plt.title('Accuracy for Each Round')
# plt.legend()
# plt.grid(True)

# fed_momentum_weighting_global_losses = [fed_momentum_weighting_model_results[round]["global_loss"] for round in fed_momentum_weighting_model_rounds]

# plt.figure(figsize=(10, 5))
# plt.plot(fed_momentum_weighting_model_rounds, fed_momentum_weighting_global_losses, marker='o', linestyle='-', color='b', label='FedPartWithVelocityWeighting')
# # plt.plot(fed_avg_model_rounds, fed_avg_global_losses, marker='o', linestyle='-', color='r', label='FedAvg')
# plt.xlabel('Round')
# plt.ylabel('Loss')
# plt.legend()
# plt.title('Loss for Each Round')